# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score #easy to calculate percentile scores
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
#this is where the process changes from equal weight alg
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 3024053155135,
 'week52high': 186.25,
 'week52low': 120.08,
 'week52highSplitAdjustOnly': 184.24,
 'week52lowSplitAdjustOnly': 119.28,
 'week52change': 0.3563440364220288,
 'sharesOutstanding': 16779915622,
 'float': 0,
 'avg10Volume': 90985942,
 'avg30Volume': 125418426,
 'day200MovingAvg': 152.48,
 'day50MovingAvg': 165.45,
 'employees': 152524,
 'ttmEPS': 11.53,
 'ttmDividendRate': 0.8853959010868717,
 'dividendYield': 0.0049089585606166735,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-27',
 'nextEarningsDate': '2022-01-24',
 'peRatio': 15.963765655699358,
 'beta': 1.368032355025019,
 'maxChangePercent': 67.3176020848036,
 'year5ChangePercent': 5.462434198298277,
 'year2ChangePercent': 1.4869323983966594,
 'year1ChangePercent': 0.3528746397849341,
 'ytdChangePercent': 0.3418115479807828,
 'month6ChangePercent': 0.3306166142517462,
 'month3ChangePercent': 0.20679682404817,
 'month1ChangePercent': 0.12489651989985194,
 'day30ChangePer

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.3528746397849341

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,165.85,0.36213,N/A
1,AAL,18.18,0.16844,N/A
2,AAP,241.10,0.479198,N/A
3,AAPL,181.86,0.354092,N/A
4,ABBV,141.08,0.367641,N/A
...,...,...,...,...
500,YUM,138.02,0.293183,N/A
501,ZBH,129.64,-0.14593,N/A
502,ZBRA,616.67,0.549911,N/A
503,ZION,65.40,0.494861,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,82.21,2.327485,N/A
1,148,DVN,45.64,1.885008,N/A
2,299,MCHP,92.00,1.63128,N/A
3,317,MRO,16.94,1.473118,N/A
4,195,FTNT,370.39,1.385497,N/A
5,345,NVDA,316.26,1.321382,N/A
6,174,F,21.60,1.299858,N/A
7,344,NUE,117.58,1.287496,N/A
8,175,FANG,115.42,1.251368,N/A
9,89,CF,74.59,1.058375,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('that is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

C:\Users\anilc\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,82.21,2.327485,2385
1,148,DVN,45.64,1.885008,4296
2,299,MCHP,92.00,1.63128,2131
3,317,MRO,16.94,1.473118,11574
4,195,FTNT,370.39,1.385497,529
5,345,NVDA,316.26,1.321382,619
6,174,F,21.60,1.299858,9077
7,344,NUE,117.58,1.287496,1667
8,175,FANG,115.42,1.251368,1698
9,89,CF,74.59,1.058375,2628


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
hqm_dataframe.dropna(inplace=True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,161,N/A,0.353103,N/A,0.075846,N/A,-0.086627,N/A,0.037999,N/A,N/A
1,AAL,18.4,N/A,0.172438,N/A,-0.18547,N/A,-0.153352,N/A,0.029008,N/A,N/A
2,AAP,240.8,N/A,0.487062,N/A,0.150646,N/A,0.081764,N/A,0.028502,N/A,N/A
3,AAPL,186.67,N/A,0.348651,N/A,0.341591,N/A,0.211422,N/A,0.125572,N/A,N/A
4,ABBV,135.32,N/A,0.36017,N/A,0.211794,N/A,0.259703,N/A,0.148349,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,144.4,N/A,0.280004,N/A,0.168301,N/A,0.084148,N/A,0.093913,N/A,N/A
501,ZBH,132.43,N/A,-0.147646,N/A,-0.236072,N/A,-0.161571,N/A,0.011344,N/A,N/A
502,ZBRA,632.18,N/A,0.56526,N/A,0.146196,N/A,0.042669,N/A,-0.014354,N/A,N/A
503,ZION,63.45,N/A,0.477614,N/A,0.144759,N/A,0.042066,N/A,-0.029611,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,161,N/A,0.353103,62.075848,0.075846,54.491018,-0.086627,11.177645,0.037999,56.087824,N/A
1,AAL,18.4,N/A,0.172438,36.726547,-0.18547,5.788423,-0.153352,3.592814,0.029008,49.9002,N/A
2,AAP,240.8,N/A,0.487062,79.241517,0.150646,69.860279,0.081764,62.674651,0.028502,49.101796,N/A
3,AAPL,186.67,N/A,0.348651,61.477046,0.341591,95.409182,0.211422,92.215569,0.125572,94.411178,N/A
4,ABBV,135.32,N/A,0.36017,63.073852,0.211794,84.431138,0.259703,94.810379,0.148349,97.005988,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,144.4,N/A,0.280004,51.896208,0.168301,75.0499,0.084148,63.672655,0.093913,86.227545,N/A
501,ZBH,132.43,N/A,-0.147646,4.790419,-0.236072,2.994012,-0.161571,3.193613,0.011344,38.123752,N/A
502,ZBRA,632.18,N/A,0.56526,85.02994,0.146196,68.862275,0.042669,46.906188,-0.014354,19.560878,N/A
503,ZION,63.45,N/A,0.477614,78.642715,0.144759,68.463074,0.042066,46.506986,-0.029611,13.772455,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,161,N/A,0.353103,62.075848,0.075846,54.491018,-0.086627,11.177645,0.037999,56.087824,45.958084
1,AAL,18.4,N/A,0.172438,36.726547,-0.18547,5.788423,-0.153352,3.592814,0.029008,49.9002,24.001996
2,AAP,240.8,N/A,0.487062,79.241517,0.150646,69.860279,0.081764,62.674651,0.028502,49.101796,65.219561
3,AAPL,186.67,N/A,0.348651,61.477046,0.341591,95.409182,0.211422,92.215569,0.125572,94.411178,85.878244
4,ABBV,135.32,N/A,0.36017,63.073852,0.211794,84.431138,0.259703,94.810379,0.148349,97.005988,84.830339
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,144.4,N/A,0.280004,51.896208,0.168301,75.0499,0.084148,63.672655,0.093913,86.227545,69.211577
501,ZBH,132.43,N/A,-0.147646,4.790419,-0.236072,2.994012,-0.161571,3.193613,0.011344,38.123752,12.275449
502,ZBRA,632.18,N/A,0.56526,85.02994,0.146196,68.862275,0.042669,46.906188,-0.014354,19.560878,55.08982
503,ZION,63.45,N/A,0.477614,78.642715,0.144759,68.463074,0.042066,46.506986,-0.029611,13.772455,51.846307


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ANET,147.01,N/A,0.985449,97.804391,0.573361,99.201597,0.594733,99.800399,0.136891,95.608782,98.103792
1,CF,75.51,N/A,1.01468,98.003992,0.424472,98.003992,0.336316,97.60479,0.152407,97.40519,97.754491
2,AVGO,677.75,N/A,0.588983,86.826347,0.474067,98.403194,0.338782,97.804391,0.224503,99.401198,95.608782
3,EXR,232.4,N/A,0.982646,97.60479,0.32951,94.810379,0.272074,95.608782,0.107813,91.217565,94.810379
4,LYV,122.39,N/A,0.666832,92.814371,0.291285,92.215569,0.291931,96.606786,0.147706,96.806387,94.610778
5,LB,80.19,N/A,2.310616,100.0,0.849275,99.800399,0.216906,93.013972,0.083164,82.834331,93.912176
6,AZO,2078.39,N/A,0.705867,94.610778,0.366491,96.407186,0.193311,90.219561,0.113671,92.015968,93.313373
7,PFE,61.7,N/A,0.644559,91.616766,0.553626,98.802395,0.362317,98.403194,0.089066,84.231537,93.263473
8,ACN,428.13,N/A,0.59049,87.225549,0.389181,97.005988,0.196416,90.419162,0.146136,96.606786,92.814371
9,PLD,170.04,N/A,0.721811,95.209581,0.342961,95.808383,0.28231,95.808383,0.085703,83.233533,92.51497


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio:10000000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

C:\Users\anilc\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\anilc\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
37,ANET,147.01,1360,0.985449,97.804391,0.573361,99.201597,0.594733,99.800399,0.136891,95.608782,98.103792
89,CF,75.51,2648,1.01468,98.003992,0.424472,98.003992,0.336316,97.60479,0.152407,97.40519,97.754491
50,AVGO,677.75,295,0.588983,86.826347,0.474067,98.403194,0.338782,97.804391,0.224503,99.401198,95.608782
173,EXR,232.4,860,0.982646,97.60479,0.32951,94.810379,0.272074,95.608782,0.107813,91.217565,94.810379
293,LYV,122.39,1634,0.666832,92.814371,0.291285,92.215569,0.291931,96.606786,0.147706,96.806387,94.610778
275,LB,80.19,2494,2.310616,100.0,0.849275,99.800399,0.216906,93.013972,0.083164,82.834331,93.912176
54,AZO,2078.39,96,0.705867,94.610778,0.366491,96.407186,0.193311,90.219561,0.113671,92.015968,93.313373
365,PFE,61.7,3241,0.644559,91.616766,0.553626,98.802395,0.362317,98.403194,0.089066,84.231537,93.263473
8,ACN,428.13,467,0.59049,87.225549,0.389181,97.005988,0.196416,90.419162,0.146136,96.606786,92.814371
373,PLD,170.04,1176,0.721811,95.209581,0.342961,95.808383,0.28231,95.808383,0.085703,83.233533,92.51497


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats ={
    'A': ['Ticker',string_template], 
    'B': ['Price', dollar_template], 
    'C': ['Number of Shares to Buy', integer_template], 
    'D': ['One-Year Price Return', percent_template], 
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [19]:
writer.save()